# Workflow #3

This notebook computes and plots priority scores for the given AOIs and centrality variants.

Important: set "save_ps_gdf" to True for updating priority score output files!

In [ ]:
import algo.net_helper as nh
import algo.helper as h 
import os.path
import geopandas as gpd
import matplotlib.pyplot as plt


# settings
aoi_names = ["at_zs", "at_wien", "at_ib", "at_no", "at_zw", "at_graz_15"]

vis_by_width = True
centrality_variants = [
    "centr_sbc_c2000_bp_d4",
    "centr_sbc_c4000_bp_d4",
    "centr_sbc_c7000_bp_d4",
    "centr_sbc_dec_2000_4000_bp_d4",
    "centr_sbc_dec_4000_7000_bp_d4",
    "centr_sbc_dec_2000_7000_bp_d4",
]
save_ps_gdf = True
plot_core_aoi=False 
plot_core_aoi_mask=True

file_suffix = ""

dir_data = "data"

mode = "bike_incwalk"
# tolerable access is determined by input network: all segments that have an index value assigned 
# (other than NULL, > 0) but have mode access set to False
access = "bicycle" 

# generated params
index_col_ft = f"index_{mode}_ft"
index_col_tf = f"index_{mode}_tf"
file = os.path.join(dir_data, f"r_<aoi_name>_edges{file_suffix}.gpkg")
file_out = os.path.join(dir_data, f"ps_<aoi_name>{file_suffix}.gpkg")
centr_cols = [f"{c}_ft" for c in centrality_variants]
centr_cols += [f"{c}_tf" for c in centrality_variants]
f_aoi = os.path.join(dir_data, "aois.gpkg")


In [ ]:
# load AOI data
aois_core = gpd.read_file(f_aoi, columns=["aoi_name", "geom"], layer="core_extent_wgs84").set_index("aoi_name")

In [ ]:
def plot_aoi(aoi, plot_core_aoi=False, plot_core_aoi_mask=True):
    print(f"\n--- processing AOI: {aoi} ---")
    fn = file.replace("<aoi_name>", aoi)
    fps = file_out.replace("<aoi_name>", aoi)
    if not os.path.exists(fn):
        print(f"ERR: file '{fn}' not found.")
        return
    # check availability of AOI extent
    if (plot_core_aoi or plot_core_aoi_mask) and (aoi not in aois_core.index.values):
        print(f"ERR: AOI extent not found in AOI table ('{aoi}')")
        return
    # read available columns
    cols = nh.read_cols_from_gpkg(fn)
    ccols = []
    for cc in centr_cols:
        if cc in cols:
            ccols.append(cc)
        else:
            print(f"WARN: centrality column '{cc}' not found for AOI '{aoi}'")
    # read available core columns and requested centrality cols
    read_cols = []
    for c in cols:
        if c.startswith("centr_"):
            continue
        read_cols.append(c)
    read_cols += ccols
    net = gpd.read_file(fn, columns=read_cols)   
    # compute priority scores
    for cname in centrality_variants:
        if not f"{cname}_ft" in cols:
            print(f"skipping unavailable centr: '{cname}'")
            continue
        centr = cname.lstrip("centr_")
        nh.compute_priorities(net, mode, centr)
        nh.compute_priorities(net, mode, centr, 0.8)
        #nh.compute_priorities(net, mode, centr, 0.6)
    if save_ps_gdf:
        print("saving priority scores to file...")
        net.to_file(fps)
        print("done.")
    # prepare output dirs
    pltdir = os.path.join("plots", aoi)
    if not os.path.exists(pltdir):
        os.makedirs(pltdir)
    # plot priority scores
    print("...plotting priority scores...")
    for col in net.columns:
        if not col.startswith("ps_") or col.endswith("_ft") or col.endswith("_tf") or (col.find("c4000")<0 and col.find("c2000")<0 and col.find("c7000")<0):
            continue
        print(col)
        net["lw"] = 0.1 + net[col] / net[col].max() * 1.5
        nf = net.loc[net[col] > 0, [col, "geometry", "lw"]]
        nf = nf.sort_values(by=col, ascending=True)
        ax = nf.plot(column=col, cmap="gist_heat_r", linewidth=nf["lw"]) # X PuRd // YlGnBu // X YlOrRd // Blues
        ## add core AOI extent
        h.set_plot_aoi(ax, aoi, plot_core_aoi, plot_core_aoi_mask)
        h.plot_map(ax, col, pltdir, dpi=1000)
        plt.show()

In [ ]:
for aoi in aoi_names:
    plot_aoi(aoi, plot_core_aoi, plot_core_aoi_mask)